<a href="https://colab.research.google.com/github/usnhee/TIL-/blob/master/function_T5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !git clone https://github.com/monologg/KoBERT-Transformers.git

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/')

In [ ]:
import tokenization_kobert

In [ ]:
from tokenization_kobert import KoBertTokenizer

In [ ]:
tokenizer_t = KoBertTokenizer.from_pretrained('monologg/kobert')

In [ ]:
!pip install tensorflow #텐서플로우 설치
!pip install sklearn #사이킷런 설치
!pip install konlpy

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
okt = Okt()
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
manhattan_d = manhattan_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])

In [ ]:
def cosin_sim(t_list, question):
  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['context'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      cos_similar = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
      t_list['score'][i] = cos_similar[0][0]
  # return cos_similar[0][0], type(cos_similar[0][0])
  return t_list.loc[t_list['score'].idxmax()]['context']

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def in_title(question, dat):
  dat = dat.drop_duplicates(['context'], ignore_index = True)
  t_list = []
  con_list = []
  for i in range(len(dat)):
    if dat['title'][i].split('(')[0] in question.replace(' ', ''):
      contet = dat.loc[i,:]
      t_list.append(contet)

    else:
      pass
  t_list = pd.DataFrame(t_list, columns = dat.columns)
  con_list = dat[dat['context'].str.contains(okt.nouns(question)[0])]
  use = pd.concat([t_list, con_list], ignore_index=True)
  if len(use) >0:
    return use
  else :
    return print('데이터 없음')


In [ ]:
def title_re(t_list, question):     

  t_list['score']=0.0
  t_list.reset_index(drop=True, inplace=True)
  for i in range(len(t_list)):
      sentences = (t_list['question'][i], question)
      tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
      tfidf_normalized = tfidf_matrix/np.sum(tfidf_matrix)
      manhattan_d = manhattan_distances(tfidf_normalized[0:1],tfidf_normalized[1:2])
      t_list['score'][i] = manhattan_d[0][0]
  t_list = t_list[t_list['score'] == min(t_list['score'])]
  return t_list


In [ ]:
def anw_test(context, question):

  source_encoding=tokenizer(
    question,
    context,
    max_length = 396,
    padding="max_length",
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)
  generated_ids = trained_model.model.generate(
    input_ids=source_encoding["input_ids"],
    attention_mask=source_encoding["attention_mask"],
    num_beams=1,  # greedy search
    max_length=80,
    repetition_penalty=2.5,
    early_stopping=True,
    use_cache=True)
  preds = [
        tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids]
  return "".join(preds), context 

In [ ]:
def last(question, dat):
  dat = dat.drop_duplicates(['context'], ignore_index = True)
  a = in_title(question,dat)   ## 타이틀, 컨텍스트 
  b= title_re(a, question)    ## 질문,타이틀 유사도 
  context =cosin_sim(b, question)    
  answer = anw_test(context, question)   ## 정답 예측, 컨텍스트 
  return answer

### 예측 정답과 실제 정답 비교용 데이터프레임 생성 


In [ ]:
sample_df = dat.sample(n = 100 , random_state=15)
re_sample = sample_df.reset_index()
ans_list = []
context_list = []

for i in range(100): 
  try:
    answer = last(re_sample.loc[i,'question'], re_sample)
    ans_list.append(answer)
    # for i in answer: 
    #   ans_list.append(a)
    #   i+=1
    #   context_list.append(a)
  except:
    ans_list.append('NaN')
    context_list.append('NaN')

In [ ]:
new_df = re_sample.copy()
for i in ans_list: 
  new_df['pred_ans'] = ans_list

new_df.drop(['answer_start','answer_end'], axis=1)